## Introducción Scraping

Trabajo de scraping sobre la web de FIFA para obtener información de los jugadores de la selección española de fútbol en el Mundial de 2022

> En este caso partimos de una página web que vamos a descargar porque se genera con javascript y no podemos obtenerla con `requests`.

https://www.fifa.com/fifaplus/es/tournaments/mens/worldcup/qatar2022/teams/spain/squad


In [ ]:
# librerías necesarias (sólo bs4 para este ejemplo) 
! pip install --upgrade pip
! pip install lxml bs4

In [ ]:
# creación del objeto BeautifulSoup para analizar el html

from bs4 import BeautifulSoup

html_doc = open('spain.html', encoding="utf-8").read()

soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
# Analizamos los contenedores de los jugadores en el html
# Vemos que están en una clase "player-badge-card_badgeCard*"
# Extracción con select

jugadores = soup.select('div[class*="player-badge-card_badgeCard"]')
print(len(jugadores))

In [ ]:
# primer jugador
j = jugadores[0]
type(j)

In [ ]:
nombre = j.select('div[class*="player-badge-card_playerFirstname"] span.d-none span')[0].text

apellido = j.select('div[class*="player-badge-card_playerSurname"] span.d-none span')[0].text

print(f'{nombre} {apellido.upper()}')

In [ ]:
# Imagen
import re
text_url = j.select('div[class*="player-badge-card_playerImage"] div')[0].attrs['style']

re.search(r'url\("(.*)"\)', text_url ).group(1)

In [ ]:
# Posición
select_posicion = 'div[class*="player-badge-card_playerPosition"] span.d-none'
j.select(select_posicion)[0].text

In [ ]:
# Todo junto

def extraer_jugador(j):
    '''Extae nombre, apellido, imagen y posición de un jugador'''
    
    nombre = j.select('div[class*="player-badge-card_playerFirstname"] span.d-none span')[0].text
    # Algunos jugadores no tienen apellido
    apellido_ = j.select('div[class*="player-badge-card_playerSurname"] span.d-none span')
    if apellido_:
        apellido = apellido_[0].text
    else:
        apellido = ''

    text_url = j.select('div[class*="player-badge-card_playerImage"] div')[0].attrs['style']
    # Algunas url no contienen la url de la imagen ¿?
    url = re.search(r'url\("(.*)"\)', text_url ).group(1)
    posicion = j.select('div[class*="player-badge-card_playerPosition"] span.d-none')[0].text

    return {'nombre': nombre, 'apellido': apellido, 'imagen': url, 'posicion': posicion}  

In [ ]:
# Extraemos la información de todos los jugadores
# Forma 1: con un bucle

datos = []
for j in jugadores:
    datos.append(extraer_jugador(j))

datos

In [ ]:
# forma 2: con list comprehension

datos = [extraer_jugador(j) for j in jugadores]

In [ ]:
# Guardamos los datos en un json
import json

with open('spain.json', 'w') as f:
    json.dump(datos, f, indent=4)
    

### ¿Y el entrenador?  Se nos ha olvidado 🤔

In [ ]:
# Todos los equpos a partir de la página de equipos la FIFA
# 

html_doc = open('equipos.html', encoding="utf-8").read()

soup = BeautifulSoup(html_doc, 'html.parser')

equipos = soup.select('div[class*="flag-grid_flagGrid__2y0Q"] a')


In [ ]:
len(equipos)